In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets, ensemble
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import datetime as dt
from sklearn import  metrics 
from sklearn.metrics import  make_scorer 
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import time
import sys

In [ ]:
# Read the datasets
train= pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
test= pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')

In [ ]:
# Check for imbalance target variable
train['Response'].value_counts()

In [ ]:
train.head()

# FEATURE ENGINEERING

In [ ]:
# Categorize Age feature

bins = [18, 30, 40, 50, 60, 70, 80, 90, 100]
lab = ['2', '3', '4', '5', '6', '7','8','9']
train['agerange'] = pd.cut(train['Age'], bins, labels = lab,include_lowest = True)
test['agerange'] = pd.cut(test['Age'], bins, labels = lab,include_lowest = True)

train['agerange']=train['agerange'].apply(np.int64)
test['agerange']=test['agerange'].apply(np.int64)

In [ ]:
train['Vehicle_Damage'].value_counts()

In [ ]:
train['Vehicle_Damage']=train['Vehicle_Damage'].map({'Yes': 10,'No': 1}).astype(int)
test['Vehicle_Damage']=test['Vehicle_Damage'].map({'Yes': 10,'No': 1}).astype(int)

train['Vehicle_Age']=train['Vehicle_Age'].map({'< 1 Year' : 0,'1-2 Year': 1,'> 2 Years': 2}).astype(int)
test['Vehicle_Age']=test['Vehicle_Age'].map({'< 1 Year' : 0,'1-2 Year': 1,'> 2 Years': 2}).astype(int)

train['Vehicle_Age'].apply(np.int64)
test['Vehicle_Age'].apply(np.int64)

train['Vehicle_Damage'].apply(np.int64)
test['Vehicle_Damage'].apply(np.int64)

train['Vehicle_Damage_Age']= train['Vehicle_Damage']*train['Vehicle_Age']
test['Vehicle_Damage_Age']= test['Vehicle_Damage']*test['Vehicle_Age']
#train['Vehicle_Damage_Age'].value_counts()

In [ ]:
train['Policy_Sales_Channel'].value_counts()

In [ ]:
train['Annual_Premium'].value_counts()

In [ ]:
train['mean_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('mean')
test['mean_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('mean')
#train['mean_annual_premium_policy_sales_channel'].value_counts()

train['max_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('max')
test['max_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('max')
#train['max_annual_premium_policy_sales_channel'].value_counts()

train['sum_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('sum')
test['sum_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('sum')
#train['sum_annual_premium_policy_sales_channel'].value_counts()

train['count_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['id'].transform('count')
test['count_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['id'].transform('count')
#train['count_annual_premium_policy_sales_channel'].value_counts()

train['min_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('min')
test['min_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('min')
#train['min_annual_premium_policy_sales_channel'].value_counts()

train['Annual_Premium_Vintage']=train['Annual_Premium']* train['Vintage']
test['Annual_Premium_Vintage']=test['Annual_Premium']* test['Vintage']

In [ ]:
train['mean_annual_premium_region_code']=train.groupby(['Region_Code'])['Annual_Premium'].transform('mean')
test['mean_annual_premium_region_code']=test.groupby(['Region_Code'])['Annual_Premium'].transform('mean')

train['max_annual_premium_region_code']=train.groupby(['Region_Code'])['Annual_Premium'].transform('max')
test['max_annual_premium_region_code']=test.groupby(['Region_Code'])['Annual_Premium'].transform('max')

train['sum_annual_premium_region_code']=train.groupby(['Region_Code'])['Annual_Premium'].transform('sum')
test['sum_annual_premium_region_code']=test.groupby(['Region_Code'])['Annual_Premium'].transform('sum')

train['count_annual_premium_region_code']=train.groupby(['Region_Code'])['id'].transform('count')
test['count_annual_premium_region_code']=test.groupby(['Region_Code'])['id'].transform('count')

train['min_annual_premium_region_code']=train.groupby(['Region_Code'])['Annual_Premium'].transform('min')
test['min_annual_premium_region_code']=test.groupby(['Region_Code'])['Annual_Premium'].transform('min')

In [ ]:
train.columns

In [ ]:
test.columns

x['Vehicle_Damage'] = x['Vehicle_Damage'].apply(np.int64)

In [ ]:
x=train.copy()
x.columns

In [ ]:
x['Gender']= x['Gender'].astype('category')
test['Gender']= test['Gender'].astype('category')

x['Vehicle_Age']= x['Vehicle_Age'].astype('category')
test['Vehicle_Age']= test['Vehicle_Age'].astype('category')

x['Vehicle_Damage']= x['Vehicle_Damage'].astype('int')
test['Vehicle_Damage']= test['Vehicle_Damage'].astype('int')

x['Driving_License']= x['Driving_License'].astype('category')
test['Driving_License']= test['Driving_License'].astype('category')

x['count_annual_premium_policy_sales_channel']= x['count_annual_premium_policy_sales_channel'].astype('category')
test['count_annual_premium_policy_sales_channel']= test['count_annual_premium_policy_sales_channel'].astype('category')

x['Policy_Sales_Channel']= x['Policy_Sales_Channel'].astype('int')
test['Policy_Sales_Channel']= test['Policy_Sales_Channel'].astype('int')


x['count_annual_premium_region_code']= x['count_annual_premium_region_code'].astype('category')
test['count_annual_premium_region_code']= test['count_annual_premium_region_code'].astype('category')

x['Previously_Insured']= x['Previously_Insured'].astype('category')
test['Previously_Insured']= test['Previously_Insured'].astype('category')

x['agerange']= x['agerange'].astype('category')
test['agerange']= test['agerange'].astype('category')

x['Region_Code']= x['Region_Code'].astype('category')
test['Region_Code']= test['Region_Code'].astype('category')


x['Response']= x['Response'].astype('category')
#test['Response']= test['Response'].astype('category')

In [ ]:
x['agerange']=x['agerange'].apply(np.int64)
test['agerange']=test['agerange'].apply(np.int64)

x['Region_Code']=x['Region_Code'].apply(np.int64)
test['Region_Code']=test['Region_Code'].apply(np.int64)

In [ ]:
c= np.unique(x['Response'])
c

In [ ]:
# Compute class weights(Imbalanced DS)
from sklearn.utils.class_weight import compute_class_weight

weigh = compute_class_weight(class_weight='balanced',classes= c,  y=x['Response'])
weigh= pd.Series(weigh)


In [ ]:
# Set up x and y(target) variables
y = x['Response']
x.drop(['id','Response'],axis=1, inplace= True)
#test.drop(['id','Response'],axis=1, inplace= True)
test_case_id= test['id']
test.drop(['id'],axis=1, inplace= True)

In [ ]:
x['Policy_Sales_Channel'].value_counts()

In [ ]:
# Set up the Voting Classifier model
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from lightgbm  import LGBMClassifier
from catboost import CatBoostClassifier

#rf = RandomForestClassifier(n_estimators=50)

categorical_features = ['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age','agerange',
                        'Region_Code', 'Policy_Sales_Channel'
                        ,'Vehicle_Damage'
                        , 'count_annual_premium_policy_sales_channel','count_annual_premium_region_code'
                       ]

rf_1 = LGBMClassifier(learning_rate=0.02, boosting_type='gbdt', max_depth=6,num_leaves = 64,   objective='binary', 
                      random_state=100, n_estimators=3000 ,
                      reg_alpha=2, reg_lambda=3, n_jobs=-1, 
                      #is_unbalance= True,
                      class_weight ={0  :  0.569842, 
                                      1  :  4.079523},
                      categorical_feature = categorical_features)

rf_2 = CatBoostClassifier(
                          learning_rate=0.01,
                          #subsample=0.085, 
                          depth=6, verbose=400,
                          bootstrap_type="Bayesian",border_count = 50, colsample_bylevel = 0.9, l2_leaf_reg = 9, 
                          cat_features = categorical_features,iterations=3000,
                          #class_weights= weigh, 
                          eval_metric='AUC',loss_function= 'Logloss',
                          bagging_temperature =  1, scale_pos_weight = 7.16
                          #random_seed': 42
                         )

rf = VotingClassifier(estimators=[('CatBoost_Best', rf_1),('LightGBM_Best', rf_2),], voting='soft',weights= [5.2,5])


In [ ]:
def timer(start_time = None):
    d= dt.datetime.now()
    if not start_time:
        start_time= d
        return start_time
    elif start_time:
        thour, temp_sec= divmod((d-start_time).total_seconds(),3600)
        tmin,tsec= divmod(temp_sec, 60)
        
        print('\n Time Taken :  %i Hours %i Minutes %i Seconds.' %(thour, tmin, round(tsec,2)))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Find accuracy over N-folds
from sklearn.model_selection import StratifiedKFold,KFold
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10)
acc = []

for fold,(t_id,v_id) in enumerate(kf.split(x,y)):
    tx = x.iloc[t_id]; ty = y.iloc[t_id]
    vx = x.iloc[v_id]; vy = y.iloc[v_id]
    start_time= timer(None)
    rf.fit(tx,ty, #weights = weights,
          )
    timer(start_time)
           
    val_y = rf.predict_proba(vx)[:,1]
    auc_score = roc_auc_score(vy,val_y)
    acc.append(auc_score)
    print(f"fold {fold} accuracy {auc_score}")

print(f"Mean accuracy score {np.mean(auc_score)}")


In [ ]:
# Make Predictions
pred= rf.predict_proba(test)[:, 1]


In [ ]:

pre= pd.Series(pred)
pre.value_counts()

In [ ]:
sub = pd.concat([test_case_id, pre], axis=1)
sub

In [ ]:
sub.rename({0: 'Response'},axis=1, inplace= True)
sub.to_csv('FinalSub.csv', index=False)

In [ ]:
'''
cols=['Gender', 'Age', 'Driving_License', 'Region_Code', 
       'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 
       'Policy_Sales_Channel', 'Vintage']
Categorical_col=['Gender','Driving_License', 
         'Region_Code', 'Previously_Insured', 
         'Vehicle_Age', 'Vehicle_Damage','Policy_Sales_Channel']
'''

In [ ]:
'''
x['Region_Code']=x['Region_Code'].astype(int)
test['Region_Code']=test['Region_Code'].astype(int)
x['Policy_Sales_Channel']=x['Policy_Sales_Channel'].astype(int)
test['Policy_Sales_Channel']=test['Policy_Sales_Channel'].astype(int)
'''

In [ ]:
#final= x[cols]

In [ ]:
'''from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final, y, test_size=.25, random_state=150400,
                                                    stratify=y,shuffle=True)
                                                    '''

In [ ]:
'''
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
model = CatBoostClassifier()
model= model.fit(X_train, y_train,cat_features=Categorical_col,eval_set=(X_test, y_test),
                 plot=True,early_stopping_rounds=30,verbose=100)
predz = model.predict(X_test)
proba_train = model.predict_proba(X_train)[:, 1]
proba_test = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_train, proba_train)
roc_auc_score(y_test, proba_test)
'''

In [ ]:
'''
pred= model.predict_proba(test[cols])[:, 1]
pre= pd.Series(pred)
pre.value_counts()
'''

In [ ]:
'''
sub = pd.concat([test_case_id, pre], axis=1)
sub.rename({0: 'Response'},axis=1, inplace= True)
sub.head()
'''

In [ ]:
#sub.to_csv('lgbm2.csv', index=False)

In [ ]:
'''
from lightgbm import LGBMClassifier
model = LGBMClassifier(n_estimators=52)
model= model.fit(X_train, y_train,eval_metric='auc',eval_set=(X_test , y_test),
                 verbose=2,categorical_feature=Categorical_col)
y_lgb = model.predict(X_test)
proba_train = model.predict_proba(X_train)[:, 1]
proba_test = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_train, proba_train)
roc_auc_score(y_test, proba_test)
'''

In [ ]:
#val_y.shape

In [ ]:
#f1= pd.Series(index= x.columns, data= rf_2.feature_importances_)
#f1.sort_values(ascending= False)[0:20][::-1].plot(kind='barh')

In [ ]:
#pred= rf_2.predict_proba(test)[:, 1]


In [ ]:
'''pre= pd.Series(pred)
pre.value_counts()
sub = pd.concat([test_case_id, pre], axis=1)
sub.rename({0: 'Response'},axis=1, inplace= True)
sub.head()
'''

In [ ]:
'''sub = pd.concat([test_case_id, pre], axis=1)
sub.rename({0: 'Response'},axis=1, inplace= True)
sub.head()
'''